In [3]:
from utils import *
import pandas as pd
import json
import re
import threading
from concurrent.futures import ThreadPoolExecutor
from threading import Lock
pd.set_option('display.max_columns', None)

In [4]:
#call openai client from utils.py
client = get_openai_client("<API_KEY>", "<BASE_URL>")

In [5]:
# Parameters
sfw_raw_data_path = "../input_data/Copy of Sfw_dataset-2023-10 (002).xlsx"
sfw_raw_data_sheet = "TSC_CCS_K&A"
target_sector = "Precision Engineering"

course_raw_data_path = "../input_data/PE TSC_Skills Heatmap_4 Apr 2024.xlsx"
course_raw_data_sheet = "2k course listing"
course_raw_data_cols = ["Unnamed: 7", "TSC Category.1", "TSC/Skill", "No. of WSQ Courses", "No. of Non-WSQ Courses", "Total No. of Courses"]

course_descr_data_path = "../input_data/precision_engineering_courses.xlsx"

course_skill_pl_output_path = "../round_2_output/courses_skill_pl.csv"

In [6]:
sfw_raw_data = pd.read_excel(sfw_raw_data_path, sheet_name=sfw_raw_data_sheet)

In [7]:
pe_knowledge_df = sfw_raw_data[sfw_raw_data["Sector"] == target_sector]
pe_knowledge_df = pe_knowledge_df.reset_index(drop=True)

pe_knowledge_df.head()

,TSC_CCS_Type,TSC_CCS Code,Sector,TSC_CCS Category,TSC_CCS Title,TSC_CCS Description,Proficiency Level,Proficiency Description,Knowledge / Ability Classification,Knowledge / Ability Items
0,tsc,PRE-RAO-1002-1.1,Precision Engineering,Automation Management,Automated Operation Monitoring,Ensure smooth automation operations by maintai...,1,Operate automated robots and system to monitor...,knowledge,Operation terminologies
1,tsc,PRE-RAO-1002-1.1,Precision Engineering,Automation Management,Automated Operation Monitoring,Ensure smooth automation operations by maintai...,1,Operate automated robots and system to monitor...,knowledge,Functions of robots and smart automated tools
2,tsc,PRE-RAO-1002-1.1,Precision Engineering,Automation Management,Automated Operation Monitoring,Ensure smooth automation operations by maintai...,1,Operate automated robots and system to monitor...,knowledge,Manufacturing execution systems
3,tsc,PRE-RAO-1002-1.1,Precision Engineering,Automation Management,Automated Operation Monitoring,Ensure smooth automation operations by maintai...,1,Operate automated robots and system to monitor...,knowledge,Application of statistical and sampling rules ...
4,tsc,PRE-RAO-1002-1.1,Precision Engineering,Automation Management,Automated Operation Monitoring,Ensure smooth automation operations by maintai...,1,Operate automated robots and system to monitor...,knowledge,Manufacturing process flows


In [8]:
pe_course_raw_data = pd.read_excel(course_raw_data_path, sheet_name = course_raw_data_sheet)
pe_course_raw_data = pe_course_raw_data.drop(columns=course_raw_data_cols)
pe_course_raw_data.head()

,TSC Category,TSC Title/Skill,Course Title,Course Reference Number,Training Provider,WSQ/Non-WSQ,Appears in 11K list?
0,Automation Management,Automated Operation Monitoring,E-Equip-Fab Autom. Proj B (SF),TGS-2018501623,GLOBALFOUNDRIES SINGAPORE PTE. LTD.,SSG-WSQ,N
1,Automation Management,Automated Operation Monitoring,E-Equip-Fab Autom. Proj D (SF),TGS-2018501625,GLOBALFOUNDRIES SINGAPORE PTE. LTD.,SSG-WSQ,N
2,Automation Management,Automated Operation Monitoring,E-Equip-Fab Autom. Proj F (SF),TGS-2018501627,GLOBALFOUNDRIES SINGAPORE PTE. LTD.,SSG-WSQ,N
3,Automation Management,Automated Operation Monitoring,E-Equip-Fab Autom. Proj G (SF),TGS-2018501628,GLOBALFOUNDRIES SINGAPORE PTE. LTD.,SSG-WSQ,N
4,Automation Management,Automated Operation Monitoring,"Ethics, Values and Legislation (Level 4)",TGS-2020505062,TSAO FOUNDATION,SSG-WSQ,N


In [9]:
pe_course_description_raw_data = pd.read_excel(course_descr_data_path)
pe_course_description_raw_data = pe_course_description_raw_data.dropna()
pe_course_description_raw_data = pe_course_description_raw_data.reset_index(drop=True)
pe_course_description_raw_data.head()

,Course Title,Course Reference Number,TP Name,TP UEN,Programme Type,About This Course,What You'll Learn
0,Food Safety Course Level 1,TGS-2018502008,COLD STORAGE SINGAPORE (1983) PTE LTD,194700005R,SSG-WSQ,"On completion, participants will have the skil...",1. Practise Good Food Hygiene_x000D_\n2. Use S...
1,Food Safety Course Level 1 Refresher,TGS-2018503461,COLD STORAGE SINGAPORE (1983) PTE LTD,194700005R,SSG-WSQ,"On completion, participants will have the skil...",1. Practise Good Food Hygiene_x000D_\n2. Use S...
2,Food Safety Course Level 1,TGS-2019504792,GOODWOOD PARK HOTEL PRIVATE LIMITED,194700135H,SSG-WSQ,To equip food handlers with the knowledge and ...,1. Practice good personal hygiene_x000D_\n2. U...
3,Practise Workplace Safety and Health for Welding,TGS-2009500052,ST ENGINEERING MARINE LTD.,196800180M,SSG-WSQ,1. Contributing to WSH programs_x000D_\n2. Res...,1. Contributing to WSH programs_x000D_\n2. Res...
4,Practise Workplace Safety & Health for Signall...,TGS-2009500231,ST ENGINEERING MARINE LTD.,196800180M,SSG-WSQ,Practise Workplace Safety & Health for Signall...,Practise Workplace Safety & Health for Signall...


In [10]:
merged_course_df = pd.merge(pe_course_description_raw_data, pe_course_raw_data, on=["Course Reference Number", "Course Title"], how="inner")
merged_course_df.head()

,Course Title,Course Reference Number,TP Name,TP UEN,Programme Type,About This Course,What You'll Learn,TSC Category,TSC Title/Skill,Training Provider,WSQ/Non-WSQ,Appears in 11K list?
0,Food Safety Course Level 1,TGS-2018502008,COLD STORAGE SINGAPORE (1983) PTE LTD,194700005R,SSG-WSQ,"On completion, participants will have the skil...",1. Practise Good Food Hygiene_x000D_\n2. Use S...,Manufacturing and Operations,Cleanliness and Contamination Control,COLD STORAGE SINGAPORE (1983) PTE LTD,SSG-WSQ,N
1,Food Safety Course Level 1 Refresher,TGS-2018503461,COLD STORAGE SINGAPORE (1983) PTE LTD,194700005R,SSG-WSQ,"On completion, participants will have the skil...",1. Practise Good Food Hygiene_x000D_\n2. Use S...,Manufacturing and Operations,Cleanliness and Contamination Control,COLD STORAGE SINGAPORE (1983) PTE LTD,SSG-WSQ,N
2,Food Safety Course Level 1,TGS-2019504792,GOODWOOD PARK HOTEL PRIVATE LIMITED,194700135H,SSG-WSQ,To equip food handlers with the knowledge and ...,1. Practice good personal hygiene_x000D_\n2. U...,Workplace Safety and Health Management,Workplace Safety and Health System Management,GOODWOOD PARK HOTEL PRIVATE LIMITED,SSG-WSQ,N
3,Practise Workplace Safety and Health for Welding,TGS-2009500052,ST ENGINEERING MARINE LTD.,196800180M,SSG-WSQ,1. Contributing to WSH programs_x000D_\n2. Res...,1. Contributing to WSH programs_x000D_\n2. Res...,Workplace Safety and Health Management,Workplace Safety and Health Practice,ST ENGINEERING MARINE LTD.,SSG-WSQ,N
4,Practise Workplace Safety and Health for Welding,TGS-2009500052,ST ENGINEERING MARINE LTD.,196800180M,SSG-WSQ,1. Contributing to WSH programs_x000D_\n2. Res...,1. Contributing to WSH programs_x000D_\n2. Res...,Workplace Safety and Health Management,Workplace Safety and Health System Management,ST ENGINEERING MARINE LTD.,SSG-WSQ,N


In [11]:
#random_sample_courses = merged_course_df["Course Reference Number"].drop_duplicates().sample(30)
#sample_course_df = merged_course_df[merged_course_df["Course Reference Number"].isin(random_sample_courses)]
#sample_course_df.head()

In [13]:
results = run_in_parallel(merged_course_df, pe_knowledge_df, client)

In [11]:
results.rename(columns={"Course Ref Number": "Course Reference Number", "Skill":"TSC Title/Skill"}, inplace=True)

In [12]:

merged_final_df = pd.merge(merged_course_df, results, on=["Course Reference Number", "TSC Title/Skill"])

In [13]:
merged_final_df.to_csv(course_skill_pl_output_path, index=False)